In [10]:
import os
os.chdir(r'\\ns-sisci01hd\home_s01\Citrix_Folder_Redirect\Downloads\SPFD')
import numpy as np
import pandas as pd
from sklearn.model_selection import ShuffleSplit
from sklearn.preprocessing import StandardScaler, MinMaxScaler, MaxAbsScaler, Normalizer
from sklearn.decomposition import PCA, TruncatedSVD

from estimators import (AverageKLPE, MaxKLPE, OCSVM,
                                       KernelSmoothing, IsolationForest, LocalOutlierFactor)
from tuning import anomaly_tuning
import time

def fillAnomaly(req_data_ref, df, scale_param, time_df, ref):
    temp1 = df.copy()
    n_estimator = 50
    N_JOBS = 1
    n_samples = df.shape[0]
    alpha_set = 0.9

    X = np.asarray(df[:])

    #algorithms = [AverageKLPE, MaxKLPE, OCSVM, IsolationForest, LocalOutlierFactor]
    #algorithms = [AverageKLPE]
    #algorithms = [IsolationForest]
    algorithms = [OCSVM]
    #algorithms = [LocalOutlierFactor]
    scaling_techniques = {'StandardScaler' :StandardScaler, 'MinMaxScaler':MinMaxScaler, 'MaxAbsScaler':MaxAbsScaler, 'Normalizer':Normalizer}
    algo_param = {'aklpe': {'k': np.arange(1, min(50, int(0.8 * n_samples)), 2),
                            'novelty': [True]},
                  'mklpe': {'k': np.arange(1, min(50, int(0.8 * n_samples)), 2),
                            'novelty': [True]},
                  'ocsvm': {'sigma': np.linspace(0.01, 5., 50)},
                  'iforest': {'max_samples': np.linspace(0.1, 1., 10)},
                  'ks': {'bandwidth': np.linspace(0.01, 5., 50)},
                  'LOF':{'n_neighbors': np.arange(1, min(50, int(0.8 * n_samples)), 2)}
                  }

    
    if scale_param :
        print('Preparocssing data with ' + scale_param)
        X = scaling_techniques[scale_param]().fit_transform(X)

    
    # Tuning step
    cv = ShuffleSplit(n_splits=n_estimator, test_size=0.2, random_state=42)
    

    
    outliers = {}
    for algo in algorithms:
        

        name_algo = algo.name
        print('--------------', name_algo, ' -------------')
        parameters = algo_param[name_algo]
        dat_length = len(X)
        start_time = time.ctime()

        models, off_set = anomaly_tuning(X, base_estimator=algo,
                                   parameters=parameters,
                                   random_state=42,
                                   cv=cv, n_jobs=N_JOBS)
        
        end_time = time.ctime()
        temp_dict = {"Data_length" :[dat_length], "Algo_name":[algo.name], "start_time":start_time, "end_time":end_time }
        temp_df =pd.DataFrame.from_dict(temp_dict) 
        time_df = pd.concat([time_df, temp_df], ignore_index =True)

        Z_data = np.zeros((n_samples,))
        
        if algo.name == 'LOF':
            for n_est in range(n_estimator):
                clf = models[n_est]
                print("Z_data_shape", Z_data.shape, "clf score sample", (clf.score_samples(X)).shape)
                Z_data += 1. / n_estimator * (clf.score_samples(X))

            off_data = np.percentile(Z_data, 100 * (1 - alpha_set))
            outliers[name_algo] = Z_data - off_data < 0

            temp =  pd.DataFrame(outliers)
            temp1 = pd.concat([req_data_ref, temp1], axis=1)
            cln = algo.name + '_scores'
            temp1[cln] = Z_data
            (pd.concat([temp1,temp], axis =1)).to_excel(name_algo+ref+'_results.xlsx', index = False)
        else:
            
            for n_est in range(n_estimator):
                clf = models[n_est]
                Z_data += 1. / n_estimator * (clf.score_samples(X))
            off_data = np.percentile(Z_data, 100 * (1 - alpha_set))
            outliers[name_algo] = Z_data - off_data < 0
            temp =  pd.DataFrame(outliers)
            temp1 = pd.concat([req_data_ref, temp1], axis=1)
            cln = algo.name + '_scores'
            temp1[cln] = Z_data
            (pd.concat([temp1, temp], axis =1)).to_excel(name_algo+ref+'_results.xlsx', index = False)
            

            
    outliers = pd.DataFrame(outliers)
    return outliers, time_df, clf, off_set, Z_data

In [11]:
res_dict = dict()
refer = '_0703_ocsvm'
df = pd.read_pickle('OMD_0703.pkl')
res, time, models, off_set, scores = fillAnomaly(req_data_ref = df[['AGENT1','YEAR']],
                        df=df.drop(['AGENT1','YEAR'],axis=1),
                       scale_param = 'StandardScaler',
                       time_df = pd.DataFrame(columns = ["Data_length", "Algo_name", "start_time", "end_time"]),
                       ref = refer)

res_dict[refer] = [res, time, models, off_set, scores]

Preparocssing data with StandardScaler
-------------- ocsvm  -------------
Dimension might be too high for volume estimation and thus deteriorate model selection
running base_estimator <class 'estimators.OCSVM'>
params {'sigma': 0.01}
0 iterations completed out of 50 for  ocsvm
params {'sigma': 0.11183673469387755}
1 iterations completed out of 50 for  ocsvm
params {'sigma': 0.21367346938775511}
2 iterations completed out of 50 for  ocsvm
params {'sigma': 0.31551020408163266}
3 iterations completed out of 50 for  ocsvm
params {'sigma': 0.41734693877551021}
4 iterations completed out of 50 for  ocsvm
params {'sigma': 0.51918367346938776}
5 iterations completed out of 50 for  ocsvm
params {'sigma': 0.62102040816326531}
6 iterations completed out of 50 for  ocsvm
params {'sigma': 0.72285714285714286}
7 iterations completed out of 50 for  ocsvm
params {'sigma': 0.82469387755102042}
8 iterations completed out of 50 for  ocsvm
params {'sigma': 0.92653061224489797}
9 iterations completed out 

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   43.4s remaining:    0.0s


0 iterations completed out of 50 for  ocsvm
params {'sigma': 0.11183673469387755}
1 iterations completed out of 50 for  ocsvm
params {'sigma': 0.21367346938775511}
2 iterations completed out of 50 for  ocsvm
params {'sigma': 0.31551020408163266}
3 iterations completed out of 50 for  ocsvm
params {'sigma': 0.41734693877551021}
4 iterations completed out of 50 for  ocsvm
params {'sigma': 0.51918367346938776}
5 iterations completed out of 50 for  ocsvm
params {'sigma': 0.62102040816326531}
6 iterations completed out of 50 for  ocsvm
params {'sigma': 0.72285714285714286}
7 iterations completed out of 50 for  ocsvm
params {'sigma': 0.82469387755102042}
8 iterations completed out of 50 for  ocsvm
params {'sigma': 0.92653061224489797}
9 iterations completed out of 50 for  ocsvm
params {'sigma': 1.0283673469387755}
10 iterations completed out of 50 for  ocsvm
params {'sigma': 1.1302040816326531}
11 iterations completed out of 50 for  ocsvm
params {'sigma': 1.2320408163265306}
12 iterations com

[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  1.4min remaining:    0.0s


0 iterations completed out of 50 for  ocsvm
params {'sigma': 0.11183673469387755}
1 iterations completed out of 50 for  ocsvm
params {'sigma': 0.21367346938775511}
2 iterations completed out of 50 for  ocsvm
params {'sigma': 0.31551020408163266}
3 iterations completed out of 50 for  ocsvm
params {'sigma': 0.41734693877551021}
4 iterations completed out of 50 for  ocsvm
params {'sigma': 0.51918367346938776}
5 iterations completed out of 50 for  ocsvm
params {'sigma': 0.62102040816326531}
6 iterations completed out of 50 for  ocsvm
params {'sigma': 0.72285714285714286}
7 iterations completed out of 50 for  ocsvm
params {'sigma': 0.82469387755102042}
8 iterations completed out of 50 for  ocsvm
params {'sigma': 0.92653061224489797}
9 iterations completed out of 50 for  ocsvm
params {'sigma': 1.0283673469387755}
10 iterations completed out of 50 for  ocsvm
params {'sigma': 1.1302040816326531}
11 iterations completed out of 50 for  ocsvm
params {'sigma': 1.2320408163265306}
12 iterations com

...

0 iterations completed out of 50 for  ocsvm
params {'sigma': 0.11183673469387755}
1 iterations completed out of 50 for  ocsvm
params {'sigma': 0.21367346938775511}
2 iterations completed out of 50 for  ocsvm
params {'sigma': 0.31551020408163266}
3 iterations completed out of 50 for  ocsvm
params {'sigma': 0.41734693877551021}
4 iterations completed out of 50 for  ocsvm
params {'sigma': 0.51918367346938776}
5 iterations completed out of 50 for  ocsvm
params {'sigma': 0.62102040816326531}
6 iterations completed out of 50 for  ocsvm
params {'sigma': 0.72285714285714286}
7 iterations completed out of 50 for  ocsvm
params {'sigma': 0.82469387755102042}
8 iterations completed out of 50 for  ocsvm
params {'sigma': 0.92653061224489797}
9 iterations completed out of 50 for  ocsvm
params {'sigma': 1.0283673469387755}
10 iterations completed out of 50 for  ocsvm
params {'sigma': 1.1302040816326531}
11 iterations completed out of 50 for  ocsvm
params {'sigma': 1.2320408163265306}
12 iterations com

0 iterations completed out of 50 for  ocsvm
params {'sigma': 0.11183673469387755}
1 iterations completed out of 50 for  ocsvm
params {'sigma': 0.21367346938775511}
2 iterations completed out of 50 for  ocsvm
params {'sigma': 0.31551020408163266}
3 iterations completed out of 50 for  ocsvm
params {'sigma': 0.41734693877551021}
4 iterations completed out of 50 for  ocsvm
params {'sigma': 0.51918367346938776}
5 iterations completed out of 50 for  ocsvm
params {'sigma': 0.62102040816326531}
6 iterations completed out of 50 for  ocsvm
params {'sigma': 0.72285714285714286}
7 iterations completed out of 50 for  ocsvm
params {'sigma': 0.82469387755102042}
8 iterations completed out of 50 for  ocsvm
params {'sigma': 0.92653061224489797}
9 iterations completed out of 50 for  ocsvm
params {'sigma': 1.0283673469387755}
10 iterations completed out of 50 for  ocsvm
params {'sigma': 1.1302040816326531}
11 iterations completed out of 50 for  ocsvm
params {'sigma': 1.2320408163265306}
12 iterations com

0 iterations completed out of 50 for  ocsvm
params {'sigma': 0.11183673469387755}
1 iterations completed out of 50 for  ocsvm
params {'sigma': 0.21367346938775511}
2 iterations completed out of 50 for  ocsvm
params {'sigma': 0.31551020408163266}
3 iterations completed out of 50 for  ocsvm
params {'sigma': 0.41734693877551021}
4 iterations completed out of 50 for  ocsvm
params {'sigma': 0.51918367346938776}
5 iterations completed out of 50 for  ocsvm
params {'sigma': 0.62102040816326531}
6 iterations completed out of 50 for  ocsvm
params {'sigma': 0.72285714285714286}
7 iterations completed out of 50 for  ocsvm
params {'sigma': 0.82469387755102042}
8 iterations completed out of 50 for  ocsvm
params {'sigma': 0.92653061224489797}
9 iterations completed out of 50 for  ocsvm
params {'sigma': 1.0283673469387755}
10 iterations completed out of 50 for  ocsvm
params {'sigma': 1.1302040816326531}
11 iterations completed out of 50 for  ocsvm
params {'sigma': 1.2320408163265306}
12 iterations com

0 iterations completed out of 50 for  ocsvm
params {'sigma': 0.11183673469387755}
1 iterations completed out of 50 for  ocsvm
params {'sigma': 0.21367346938775511}
2 iterations completed out of 50 for  ocsvm
params {'sigma': 0.31551020408163266}
3 iterations completed out of 50 for  ocsvm
params {'sigma': 0.41734693877551021}
4 iterations completed out of 50 for  ocsvm
params {'sigma': 0.51918367346938776}
5 iterations completed out of 50 for  ocsvm
params {'sigma': 0.62102040816326531}
6 iterations completed out of 50 for  ocsvm
params {'sigma': 0.72285714285714286}
7 iterations completed out of 50 for  ocsvm
params {'sigma': 0.82469387755102042}
8 iterations completed out of 50 for  ocsvm
params {'sigma': 0.92653061224489797}
9 iterations completed out of 50 for  ocsvm
params {'sigma': 1.0283673469387755}
10 iterations completed out of 50 for  ocsvm
params {'sigma': 1.1302040816326531}
11 iterations completed out of 50 for  ocsvm
params {'sigma': 1.2320408163265306}
12 iterations com

[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed: 36.1min finished
